In [2]:
import pandas as pd
import numpy as np

In [3]:
df_may = pd.read_csv("KRI-DAC_Jeju_data5.txt")


In [134]:
df_june = pd.read_csv("KRI-DAC_Jeju_data6.txt")
df_july = pd.read_csv("KRI-DAC_Jeju_data7.txt")
df_aug = pd.read_csv("KRI-DAC_Jeju_data8.txt")

YM : 기준년월

SIDO : 지역대분류명

SIGUNGU : 지역중분류명

FranClass : 소상공인구분

Type : 업종명

Time : 시간대

TotalSpent : 총사용금액

DisSpent : 재난지원금 사용금액

NumOfSpent : 총 이용건수

NumOfDisSpent : 총 재난지원금 이용건수

POINT_X, POINT_Y : X,Y 좌표

시간대 : 2-6 : toqur, 6-11 : 오전, 11-15 : 점심, 15-18: 오후, 18-22 : 저녁, 22-02 : 심야 x : 무승인거래

### 각 월별로 데이터가 분리되어있음

## 월 별 결측치 확인

In [88]:
df_may.isnull().sum()

OBJECTID         0
Field1           0
YM               0
SIDO             0
SIGUNGU          0
FranClass        0
Type             0
Time             0
TotalSpent       0
DisSpent         0
NumofSpent       0
NumofDisSpent    0
POINT_X          0
POINT_Y          0
lat              0
long             0
address          0
dtype: int64

In [89]:
df_june.isnull().sum()

OBJECTID         0
Field1           0
YM               0
SIDO             0
SIGUNGU          0
FranClass        0
Type             0
Time             0
TotalSpent       0
DisSpent         0
NumofSpent       0
NumofDisSpent    0
POINT_X          0
POINT_Y          0
lat              0
long             0
dtype: int64

In [90]:
df_july.isnull().sum()

OBJECTID         0
Field1           0
YM               0
SIDO             0
SIGUNGU          0
X                0
Y                0
FranClass        0
Type             0
Time             0
TotalSpent       0
DisSpent         0
NumofSpent       0
NumofDisSpent    0
POINT_X          0
POINT_Y          0
lat              0
long             0
dtype: int64

In [91]:
df_aug.isnull().sum()

OBJECTID         0
Field1           0
YM               0
SIDO             0
SIGUNGU          0
FranClass        0
Type             0
Time             0
TotalSpent       0
DisSpent         0
NumofSpent       0
NumofDisSpent    0
POINT_X          0
POINT_Y          0
lat              0
long             0
dtype: int64

## X,Y좌표를 위도 경도로 변환

In [4]:
pip install pyproj

Note: you may need to restart the kernel to use updated packages.


In [5]:
from pyproj import Proj,Transformer

In [6]:
#UTM-K(Bessel) : 새주소지도에서 사용중
#한반도 전체를 하나으 ㅣ좌표계로 나타낼 때 많이 사용하는 좌표계
#proj_UTMK = Proj(init = 'epsg:5178')

#WGS1984: GPS가 사용하는 좌표계
#proj_WGS84 = Proj(init = 'epsg:4326')
transformer = Transformer.from_crs('epsg:5179', 'epsg:4326')
#transformer.transform(12, 12)
#UTM-K -> WGS84
def transform_utmk_to_w84(x, y):
    return pd.Series(transformer.transform(x, y))

In [7]:
def add_long_lat_feature(df):
    df_long_lat = df.apply(lambda x :transform_utmk_to_w84(x.POINT_Y, x.POINT_X), axis =1)
    df_long_lat.columns = ['lat', 'long']
    return pd.concat([df, df_long_lat], axis = 1)

In [8]:
df_may = add_long_lat_feature(df_may)

In [136]:
df_june = add_long_lat_feature(df_june)

In [160]:
df_july = add_long_lat_feature(df_july)
df_aug = add_long_lat_feature(df_aug)

In [64]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


## 카카오 API를 통하여 위도경도 정보를 주소로 변환

In [65]:
import requests
import json

In [66]:
APP_KEY = "2729ea07c6c8d521d0fde6d8c164f58d"
URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'

In [68]:
def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': 'KakaoAK {}'.format(APP_KEY)}
    resp = requests.get(url, headers=headers)
    #print('%s : success for request [%s]' % (datetime.now(), url))
    return resp.text


def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[0]
        json_name = json_doc.get('address_name')

        #json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name #json_code

In [69]:
spatio_data = df_may.groupby(['lat', 'long'])['OBJECTID'].count().sort_values(ascending = False).reset_index()
spatio_data.rename(columns = {'OBJECTID' : 'count'})

,lat,long,count
0,33.512906,126.524279,400
1,33.498992,126.529357,314
2,33.499887,126.528804,309
3,33.485749,126.477935,275
4,33.247397,126.563626,258
...,...,...,...
13815,33.484292,126.433310,1
13816,33.484279,126.432234,1
13817,33.484245,126.429545,1
13818,33.484198,126.461823,1


In [72]:
list_address = []
for i in range(len(spatio_data)):
    address = reverse_geocode(spatio_data['long'][i], spatio_data['lat'][i])
    list_address.insert(i, address)

In [74]:
df_address = pd.DataFrame({'address' : list_address})
sptio_data = pd.concat([spatio_data, df_address], axis = 1)

In [83]:
df_may = pd.merge(df_may, sptio_data[['lat', 'long', 'address']],  how = 'left', on = ['lat', 'long'])

In [85]:
df_may.isnull().sum()

OBJECTID         0
Field1           0
YM               0
SIDO             0
SIGUNGU          0
FranClass        0
Type             0
Time             0
TotalSpent       0
DisSpent         0
NumofSpent       0
NumofDisSpent    0
POINT_X          0
POINT_Y          0
lat              0
long             0
address          0
dtype: int64

결측치가 없음을 확인

In [138]:
spatio_data

,lat,long,OBJECTID
0,33.512906,126.524279,400
1,33.498992,126.529357,314
2,33.499887,126.528804,309
3,33.485749,126.477935,275
4,33.247397,126.563626,258
...,...,...,...
13815,33.484292,126.433310,1
13816,33.484279,126.432234,1
13817,33.484245,126.429545,1
13818,33.484198,126.461823,1


In [139]:
spatio_data = df_may.groupby(['lat', 'long', 'address'])['OBJECTID'].count().sort_values(ascending = False).reset_index()

In [142]:
df_june = pd.merge(df_june, spatio_data[['lat', 'long', 'address']],  how = 'left', on = ['lat', 'long'])

In [145]:
df_june.isnull().sum()

OBJECTID            0
Field1              0
YM                  0
SIDO                0
SIGUNGU             0
FranClass           0
Type                0
Time                0
TotalSpent          0
DisSpent            0
NumofSpent          0
NumofDisSpent       0
POINT_X             0
POINT_Y             0
lat                 0
long                0
address          1796
dtype: int64

5,6,7,8이 되면서 새로운 좌표도 점차 생긴 것을 확인해볼 수 있다

In [147]:
june_spatio = df_june[df_june['address'].isnull()].groupby(['lat', 'long'])['OBJECTID'].count().sort_values(ascending = False).reset_index()

In [148]:
june_spatio = june_spatio.rename(columns = {'OBJECTID' : 'count'})

In [149]:
list_address = []
for i in range(len(june_spatio)):
    address = reverse_geocode(june_spatio['long'][i], june_spatio['lat'][i])
    list_address.insert(i, address)

In [150]:
df_address = pd.DataFrame({'address' : list_address})
june_spatio = pd.concat([june_spatio, df_address], axis = 1)

In [153]:
df_empty_june = pd.merge(df_june[df_june['address'].isnull()].drop(['address'], axis = 1), june_spatio[['lat', 'long', 'address']],  how = 'left', on = ['lat', 'long'])

In [155]:
df_june = df_june.drop(df_june[df_june['address'].isnull()].index, axis = 0)

In [157]:
df_june = pd.concat([df_june, df_empty_june], axis = 0)

In [159]:
df_june.isnull().sum()

OBJECTID         0
Field1           0
YM               0
SIDO             0
SIGUNGU          0
FranClass        0
Type             0
Time             0
TotalSpent       0
DisSpent         0
NumofSpent       0
NumofDisSpent    0
POINT_X          0
POINT_Y          0
lat              0
long             0
address          0
dtype: int64

In [161]:
spatio_data = df_june.groupby(['lat', 'long', 'address'])['OBJECTID'].count().sort_values(ascending = False).reset_index()

In [162]:
df_july = pd.merge(df_july, spatio_data[['lat', 'long', 'address']],  how = 'left', on = ['lat', 'long'])

In [163]:
july_spatio = df_july[df_july['address'].isnull()].groupby(['lat', 'long'])['OBJECTID'].count().sort_values(ascending = False).reset_index()

In [164]:
july_spatio = july_spatio.rename(columns = {'OBJECTID' : 'count'})

In [166]:
list_address = []
for i in range(len(july_spatio)):
    address = reverse_geocode(july_spatio['long'][i], july_spatio['lat'][i])
    list_address.insert(i, address)

In [167]:
df_address = pd.DataFrame({'address' : list_address})
july_spatio = pd.concat([july_spatio, df_address], axis = 1)

In [168]:
df_empty_july = pd.merge(df_july[df_july['address'].isnull()].drop(['address'], axis = 1), july_spatio[['lat', 'long', 'address']],  how = 'left', on = ['lat', 'long'])

In [169]:
df_july = df_july.drop(df_july[df_july['address'].isnull()].index, axis = 0)

In [170]:
df_july = pd.concat([df_july, df_empty_july], axis = 0)

In [173]:
df_july.isnull().sum()

OBJECTID         0
Field1           0
YM               0
SIDO             0
SIGUNGU          0
X                0
Y                0
FranClass        0
Type             0
Time             0
TotalSpent       0
DisSpent         0
NumofSpent       0
NumofDisSpent    0
POINT_X          0
POINT_Y          0
lat              0
long             0
address          0
dtype: int64

In [174]:
spatio_data = df_july.groupby(['lat', 'long', 'address'])['OBJECTID'].count().sort_values(ascending = False).reset_index()

In [175]:
df_aug = pd.merge(df_aug, spatio_data[['lat', 'long', 'address']],  how = 'left', on = ['lat', 'long'])

In [176]:
aug_spatio = df_aug[df_aug['address'].isnull()].groupby(['lat', 'long'])['OBJECTID'].count().sort_values(ascending = False).reset_index()

In [177]:
aug_spatio = aug_spatio.rename(columns = {'OBJECTID' : 'count'})

In [178]:
list_address = []
for i in range(len(aug_spatio)):
    address = reverse_geocode(aug_spatio['long'][i], aug_spatio['lat'][i])
    list_address.insert(i, address)

In [179]:
df_address = pd.DataFrame({'address' : list_address})
aug_spatio = pd.concat([aug_spatio, df_address], axis = 1)

In [180]:
df_empty_aug = pd.merge(df_aug[df_aug['address'].isnull()].drop(['address'], axis = 1), aug_spatio[['lat', 'long', 'address']],  how = 'left', on = ['lat', 'long'])

In [181]:
df_aug = df_aug.drop(df_aug[df_aug['address'].isnull()].index, axis = 0)

In [182]:
df_aug = pd.concat([df_aug, df_empty_aug], axis = 0)

In [183]:
df_aug

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lat,long,address
0,1,1,202008,제주특별자치도,제주시,영세,일반한식,00시,249500,0,7,0,877005.9834,1.479766e+06,33.302315,126.178894,제주특별자치도 제주시 한경면 고산리
1,2,2,202008,제주특별자치도,제주시,영세,단란주점,00시,2010000,0,7,0,877005.7447,1.479816e+06,33.302766,126.178884,제주특별자치도 제주시 한경면 고산리
2,3,3,202008,제주특별자치도,제주시,중소1,편의점,00시,401210,0,41,0,877056.6756,1.479616e+06,33.300971,126.179458,제주특별자치도 제주시 한경면 고산리
3,4,4,202008,제주특별자치도,제주시,영세,편의점,00시,12250,0,1,0,877055.9593,1.479766e+06,33.302323,126.179430,제주특별자치도 제주시 한경면 고산리
4,5,5,202008,제주특별자치도,제주시,영세,주점,00시,33000,0,1,0,877055.4817,1.479866e+06,33.303224,126.179412,제주특별자치도 제주시 한경면 고산리
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585,279501,279501,202008,제주특별자치도,제주시,일반,서양음식,x시,10000,0,1,0,936031.5193,1.500490e+06,33.494351,126.811369,제주특별자치도 제주시 구좌읍 평대리
1586,279728,279728,202008,제주특별자치도,제주시,일반,서양음식,x시,2500,0,1,0,940163.7221,1.503859e+06,33.524971,126.855625,제주특별자치도 제주시 구좌읍 세화리
1587,279785,279785,202008,제주특별자치도,제주시,영세,기타숙박업,x시,17470,0,2,0,941114.0531,1.503713e+06,33.523713,126.865869,제주특별자치도 제주시 구좌읍 하도리
1588,279811,279811,202008,제주특별자치도,서귀포시,영세,제과점,x시,40000,0,1,0,942706.5993,1.494724e+06,33.442728,126.883592,제주특별자치도 서귀포시 성산읍 수산리


In [184]:
df_may.to_csv("data_may.csv")
df_june.to_csv("data_june.csv")
df_july.to_csv("data_july.csv")
df_aug.to_csv("data_aug.csv")

## 월별 지역의 unique 갯수

In [188]:
print(df_may['address'].nunique())
print(df_june['address'].nunique())
print(df_july['address'].nunique())
print(df_aug['address'].nunique())

184
185
185
185


5월에 비해 추가된 지역은 1곳이다

In [192]:
set(df_july['address'].unique()) - set(df_may['address'].unique())

{'제주특별자치도 제주시 추자면 묵리'}

In [194]:
df_june[df_june['address'] == '제주특별자치도 제주시 추자면 묵리']

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lat,long,address
168,42616,42616,202006,제주특별자치도,제주시,영세,기타숙박업,10시,140000,0,1,0,890407.0176,1550898.51,33.945202,126.314052,제주특별자치도 제주시 추자면 묵리


In [193]:
df_july[df_july['address'] == '제주특별자치도 제주시 추자면 묵리']

,OBJECTID,Field1,YM,SIDO,SIGUNGU,X,Y,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lat,long,address
1654,226564,226564,202007,제주특별자치도,제주시,244400,151050,영세,기타숙박업,20시,50000,0,1,0,890457.2371,1.550849e+06,33.944759,126.314601,제주특별자치도 제주시 추자면 묵리


In [195]:
df_aug[df_aug['address'] == '제주특별자치도 제주시 추자면 묵리']

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lat,long,address
59,25180,25180,202008,제주특별자치도,제주시,영세,기타숙박업,08시,465000,0,1,0,890407.0176,1550898.51,33.945202,126.314052,제주특별자치도 제주시 추자면 묵리


재난 지원금 사용 x 그리고 월별 1번만 이용하는 것을 보임 이는 전체적인 경향을 보기에 어려운 데이터라고 할 수 있음(이상치)

## 월별 지역간 업종 빈도수

In [198]:
df_may['Type'].unique()

array(['일반한식', '단란주점', '편의점', '주점', '스넥', '서양음식', '일식회집', '기타음료식품', '노래방',
       '중국음식', '슈퍼마켓', '유흥주점', '기타숙박업', '기타레져업', '당구장', '농축수산품', '유아원',
       '골프경기장', '세탁소', '가방', '콘도', '주차장', '문화취미기타', '주유소', '사무서비스',
       '자동차정비', '특급호텔', '볼링장', '화장품', '기타회원제형태업소', '주류판매점', '제과점',
       '피부미용실', '약국', '칵테일바', '정육점', '독서실', '스포츠레져용품', '골프용품', 'LPG',
       '종합병원', '사우나', '화원', '의원', '안마스포츠마사지', '골프연습장', '2급호텔', '기타대인서비스',
       '보습학원', '부동산분양', '화물운송', '애완동물', '동물병원', '대형할인점', '인터넷Mall', '항공사',
       '헬스크럽', '기타유통업', '레져용품수리', '정장', '레져업소(회원제형태)', '관광여행', '1급호텔',
       '침구수예점', '사진관', '악세사리', '보관창고업', '기념품점', '미용원', '영화관', '의료용품',
       '세차장', '인터넷종합Mall', '기타잡화', '성인용품점', '상품권', '종합용역', '기계공구',
       '조세서비스', '택시', '기타의료기관및기기', '가례서비스', '구내매점', '수영장', '목재석재철물',
       '건축요업품', '기타업종', '렌트카', '종합레져타운', '유류판매', '농축협직영매장', '기타건축자재',
       '비료농약사료종자', '농기계', '보일러펌프', '가전제품', '스포츠의류', '페인트', '미곡상',
       '기타농업관련', '기타가구', '공공요금', '기능학원', '면세점', '기타교육', '위탁급식업', '이용원',
       '조명기구',

In [206]:
df_may.groupby(['address','Type'])['OBJECTID'].count().sort_values(ascending = False).reset_index().head(10)

,address,Type,OBJECTID
0,제주특별자치도 제주시 연동,일반한식,5452
1,제주특별자치도 제주시 노형동,일반한식,4640
2,제주특별자치도 제주시 이도이동,일반한식,4421
3,제주특별자치도 서귀포시 서귀동,일반한식,3074
4,제주특별자치도 제주시 연동,편의점,2250
5,제주특별자치도 제주시 일도이동,일반한식,2117
6,제주특별자치도 제주시 노형동,편의점,1960
7,제주특별자치도 제주시 노형동,서양음식,1797
8,제주특별자치도 제주시 이도이동,서양음식,1445
9,제주특별자치도 제주시 이도이동,편의점,1425


In [210]:
df_june.groupby(['address','Type'])['OBJECTID'].count().sort_values(ascending = False).reset_index().head(10)

,address,Type,OBJECTID
0,제주특별자치도 제주시 연동,일반한식,5734
1,제주특별자치도 제주시 노형동,일반한식,4782
2,제주특별자치도 제주시 이도이동,일반한식,4542
3,제주특별자치도 서귀포시 서귀동,일반한식,3215
4,제주특별자치도 제주시 연동,편의점,2293
5,제주특별자치도 제주시 일도이동,일반한식,2229
6,제주특별자치도 제주시 노형동,편의점,1961
7,제주특별자치도 제주시 노형동,서양음식,1899
8,제주특별자치도 제주시 이도이동,서양음식,1520
9,제주특별자치도 서귀포시 동홍동,일반한식,1464


In [207]:
df_july.groupby(['address','Type'])['OBJECTID'].count().sort_values(ascending = False).reset_index().head(10)

,address,Type,OBJECTID
0,제주특별자치도 제주시 연동,일반한식,5705
1,제주특별자치도 제주시 노형동,일반한식,4906
2,제주특별자치도 제주시 이도이동,일반한식,4526
3,제주특별자치도 서귀포시 서귀동,일반한식,3170
4,제주특별자치도 제주시 연동,편의점,2308
5,제주특별자치도 제주시 일도이동,일반한식,2217
6,제주특별자치도 제주시 노형동,편의점,1977
7,제주특별자치도 제주시 노형동,서양음식,1913
8,제주특별자치도 제주시 이도이동,서양음식,1522
9,제주특별자치도 제주시 조천읍 함덕리,일반한식,1478


In [208]:
df_aug.groupby(['address','Type'])['OBJECTID'].count().sort_values(ascending = False).reset_index().head(10)

,address,Type,OBJECTID
0,제주특별자치도 제주시 연동,일반한식,5567
1,제주특별자치도 제주시 노형동,일반한식,4848
2,제주특별자치도 제주시 이도이동,일반한식,4349
3,제주특별자치도 서귀포시 서귀동,일반한식,3072
4,제주특별자치도 제주시 연동,편의점,2339
5,제주특별자치도 제주시 일도이동,일반한식,2128
6,제주특별자치도 제주시 노형동,편의점,1982
7,제주특별자치도 제주시 노형동,서양음식,1888
8,제주특별자치도 제주시 조천읍 함덕리,일반한식,1527
9,제주특별자치도 제주시 이도이동,서양음식,1514


- 각 동에서의 가게의 빈도 수는 변함이 거의 없음
- 일반 한식의 비중이 상당함
- 서양음식과 편의점의 빈도도 높은편

연동, 노형동, 이도이동, 일도이동이 핫플인것인가?

In [201]:
df_june[df_june['NumofSpent'] == 0]

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lat,long,address


In [199]:
df_may[(df_may['address'] == "제주특별자치도 제주시 연동") & (df_may['Type'] == "일반한식") ]

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lat,long,address
1052,1053,1053,202005,제주특별자치도,제주시,중소,일반한식,00시,160500,0,4,0,905294.5896,1.500392e+06,33.491191,126.480530,제주특별자치도 제주시 연동
1117,1118,1118,202005,제주특별자치도,제주시,영세,일반한식,00시,123000,53000,4,2,905544.7169,1.500344e+06,33.490773,126.483227,제주특별자치도 제주시 연동
1118,1119,1119,202005,제주특별자치도,제주시,일반,일반한식,00시,99000,0,4,0,905544.4770,1.500394e+06,33.491223,126.483219,제주특별자치도 제주시 연동
1215,1216,1216,202005,제주특별자치도,제주시,영세,일반한식,00시,23600,23600,1,1,905743.1872,1.500645e+06,33.493503,126.485332,제주특별자치도 제주시 연동
1230,1231,1231,202005,제주특별자치도,제주시,영세,일반한식,00시,235000,0,5,0,905846.7414,1.499895e+06,33.486756,126.486525,제주특별자치도 제주시 연동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270378,270379,270379,202005,제주특별자치도,제주시,중소,일반한식,x시,16000,0,1,0,906999.3436,1.499251e+06,33.481047,126.498997,제주특별자치도 제주시 연동
270389,270390,270390,202005,제주특별자치도,제주시,영세,일반한식,x시,30000,0,1,0,907048.8414,1.499351e+06,33.481955,126.499519,제주특별자치도 제주시 연동
270390,270391,270391,202005,제주특별자치도,제주시,중소1,일반한식,x시,35000,0,2,0,907047.1621,1.499701e+06,33.485110,126.499465,제주특별자치도 제주시 연동
270401,270402,270402,202005,제주특별자치도,제주시,중소1,일반한식,x시,30000,0,1,0,907099.2988,1.499252e+06,33.481060,126.500073,제주특별자치도 제주시 연동


새로 생성한 address를 이용하여 지역을 구분할 수 있다.